# Day 19: Beacon Scanner

2D example.

In [1]:
example = """--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1"""

In [2]:
def parse(input):
    """Parses beacon locations reported by scanners from input."""
    return tuple(
        set(
            # Beacon positions can just be eval'd to tuples.
            eval(line)
            # Discard the first line of each scanner report.
            for line in report.splitlines()[1:]
        )
        # Scanner reports are separated by two newlines.
        for report in input.split('\n\n')
    )

parse(example)

({(0, 2), (3, 3), (4, 1)}, {(-5, 0), (-2, 1), (-1, -1)})

I think distance between beacons in a scanner report will be useful because distances are the same regardless of scanner position and orientation.

In [3]:
import math

def distance(a, b):
    """Returns Euclidean distance between n-dimensional points a and b."""
    return math.sqrt(sum((a_n - b_n)**2 for a_n, b_n in zip(a, b))) 

# 2D points.
print(distance((0, 0), (1, 1)))
# 3D points.
print(distance((0, 0, 0), (1, 1, 1)))

1.4142135623730951
1.7320508075688772


In [4]:
import itertools

def distances(points):
    """Returns set of distances between pairs of points."""
    return {distance(a, b) for a, b in itertools.combinations(points, r=2)}

distances(parse(example)[0])

{2.23606797749979, 3.1622776601683795, 4.123105625617661}

In this example all scanners see the same beacons, but with different orientations.

In [5]:
example2 = """--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8"""

In this case, distances should be equal for all reports.

In [6]:
# The set of all distance sets should contain only one element!
len(set(frozenset(distances(points)) for points in parse(example2)))

1

In [7]:
example3 = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

Build graph of overlapping scanners based on max shared distances.

In [10]:
report = parse(example3)

def calc_overlaps(scanners):
    """Returns graph (dict) of overlapping scanner indices."""
    # Graph of scanner overlaps.
    overlaps = {}
    unvisited = set(range(len(scanners)))
    unvisited.remove(0)
    visited = {0}

    while unvisited:
        # Determine which pair of scanners a and b have the greatest
        # number of shared distances. We can assume b overlaps with a.
        _, a, b = max(
            (len(distances(scanners[a]) & distances(scanners[b])), a, b)
            for a in visited
            for b in unvisited
        )
        print(b, 'overlaps with', a)
        overlaps[b] = a
        unvisited.remove(b)
        visited.add(b)
    
overlaps = calc_overlaps(report)
overlaps

1 overlaps with 0
4 overlaps with 1
2 overlaps with 4
3 overlaps with 1


want a function that takes a point from scanner a, and returns it's position relative to scanner b.

- Consider x components of scanner 0 and scanner 1
- The x co-ordinate *could* be the same for each scanner
- But it's unlikely the scanners are in the same position
- So we'll translate scanner 1 by different amounts and calculate the amount of correlation with scanner 0
- Start by translating scanner 1 by the min of scanner 0 minus the max of scanner 1
- This would shift scanner 1 from this:
    ```
                     ---scanner 0 x--->
                                 ---scanner 1 x--->
    ```

    To this:
    ```
                     ---scanner 0 x--->
    ---scanner 1 x--->
    ```
- Increment translation until it is equal to the max of scanner 0 x minus the min of scanner 1 x
    ```
                     ---scanner 0 x--->
                                      ---scanner 1 x--->
    ```
- Then we'll record the translation that results in greatest correlation/overlap

In [312]:
def create_transform(ref, other):
    """Returns a function that transforms points in other to ref."""
    # Extract x, y, z components from points.
    other_components = tuple(set(component) for component in zip(*other))
    ref_components = tuple(set(component) for component in zip(*ref))

    # Maps ref dimensions to other dimensions to ref dimensions. 
    dimension_map = {}
    
    # Maps ref dimension to function 
    transform_map = {}
    
#     info_map = {}
    
    other_dimensions = set(range(3))
    
    # Reference dimensions.
    for i in range(3):
        
        _, j, flipped, offset = max(
            (
                # Count overlaps of a and b translated by each offset.
                len(
                    ref_components[i] & 
                    {(-x if flipped else x) + offset for x in other_components[j]}
                ),
                j,
                flipped,
                offset
            )
            for j in other_dimensions
            # Consider the other dimension flipped or not.
            for flipped in (True, False)
            # Range of offsets applied to other that have any possibility 
            # of overlapping with ref.
            for offset in range(
                min(ref_components[i]) - max({-x if flipped else x for x in other_components[j]}), 
                max(ref_components[i]) - min({-x if flipped else x for x in other_components[j]})
            )
        )
        
        dimension_map[i] = j
        transform_map[i] = lambda x: (-x if flipped else x) + offset
        info_map[i] = {'flipped': flipped, 'offset': offset, 'other': j}
        other_dimensions.remove(j)
        print(f'{j} maps to {i}, {flipped=}, {offset=}')
        
    def transform(point):
        transformed_point = []
        
        for dim in range(len(point)):
#             print(f'{dim} maps to {info_map[dim]}')
            
            x = point[info_map[dim]['other']]
            flipped = info_map[dim]['flipped']
            offset = info_map[dim]['offset']
            transformed_point.append((-x if flipped else x) + offset)
        return tuple(transformed_point)
            
    return transform
#     return lambda point: tuple(transform_map[i](point[dimension_map[i]]) for i in range(len(point)))

In [313]:
tf = create_transform(report[1], report[0])

0 maps to 0, flipped=True, offset=68
1 maps to 1, flipped=False, offset=-1246
2 maps to 2, flipped=True, offset=-43


In [326]:
transforms = {}

In [328]:
for scanner in overlaps:
    transforms[scanner] = create_transform(report[scanner], report[overlaps[scanner]])

0 maps to 0, flipped=True, offset=68
1 maps to 1, flipped=False, offset=-1246
2 maps to 2, flipped=True, offset=-43
1 maps to 0, flipped=False, offset=88
2 maps to 1, flipped=True, offset=113
0 maps to 2, flipped=True, offset=-1104
1 maps to 0, flipped=False, offset=168
0 maps to 1, flipped=False, offset=-1125
2 maps to 2, flipped=True, offset=72
0 maps to 0, flipped=False, offset=160
1 maps to 1, flipped=False, offset=-1134
2 maps to 2, flipped=False, offset=-23


In [329]:
transforms

{1: <function __main__.create_transform.<locals>.transform(point)>,
 4: <function __main__.create_transform.<locals>.transform(point)>,
 2: <function __main__.create_transform.<locals>.transform(point)>,
 3: <function __main__.create_transform.<locals>.transform(point)>}

In [319]:
report[0] & {tf(point) for point in report[1]} == {
    eval(line) for line in """-618,-824,-621
-537,-823,-458
-447,-329,318
404,-588,-901
544,-627,-890
528,-643,409
-661,-816,-575
390,-675,-793
423,-701,434
-345,-311,381
459,-707,401
-485,-357,347""".splitlines()
}

True

In [320]:
overlaps

{1: 0, 4: 1, 2: 4, 3: 1}

What do we actually want? Beacons seen by all scanners with co-ordinates relative to scanner 0.

In [336]:
def gen_points(report):
    """Generate points from report with co-ordinates relative to scanner 0."""
    for scanner, points in enumerate(report):
        points = iter(points)
        
        # Move through overlaps graph until we get to scanner 0.
        while scanner:
            print(f'{scanner=}')
            points = [transforms[scanner](point) for point in points]
            scanner = overlaps[scanner]
        yield from points

This result lines up with the example.

In [337]:
beacons = set(gen_points(parse(example3)))
len(beacons)

scanner=1
scanner=2
scanner=4
scanner=1
scanner=3
scanner=1
scanner=4
scanner=1


79

Try it on puzzle input.

In [338]:
report = parse(open('day-19-input.txt').read())

# Graph of scanner overlaps.
overlaps = {}
unvisited = set(range(len(report)))
unvisited.remove(0)
visited = {0}

while unvisited:
    # Determine which pair of scanners a and b have the greatest
    # number of shared distances. We can assume b overlaps with a.
    _, a, b = max(
        (len(distances(report[a]) & distances(report[b])), a, b)
        for a in visited
        for b in unvisited
    )
    print(b, 'overlaps with', a)
    overlaps[b] = a
    unvisited.remove(b)
    visited.add(b)
    
overlaps

24 overlaps with 0
18 overlaps with 24
16 overlaps with 24
13 overlaps with 18
2 overlaps with 18
5 overlaps with 16
15 overlaps with 13
17 overlaps with 15
23 overlaps with 17
11 overlaps with 23
1 overlaps with 23
10 overlaps with 1
21 overlaps with 17
4 overlaps with 21
14 overlaps with 17
12 overlaps with 15
22 overlaps with 14
9 overlaps with 22
3 overlaps with 13
8 overlaps with 12
7 overlaps with 11
6 overlaps with 11
20 overlaps with 7
19 overlaps with 1


{24: 0,
 18: 24,
 16: 24,
 13: 18,
 2: 18,
 5: 16,
 15: 13,
 17: 15,
 23: 17,
 11: 23,
 1: 23,
 10: 1,
 21: 17,
 4: 21,
 14: 17,
 12: 15,
 22: 14,
 9: 22,
 3: 13,
 8: 12,
 7: 11,
 6: 11,
 20: 7,
 19: 1}

In [341]:
transforms = {}

In [342]:
for scanner in overlaps:
    transforms[scanner] = create_transform(report[scanner], report[overlaps[scanner]])

2 maps to 0, flipped=False, offset=-62
0 maps to 1, flipped=True, offset=-53
1 maps to 2, flipped=True, offset=1206
1 maps to 0, flipped=False, offset=-122
2 maps to 1, flipped=True, offset=-1041
0 maps to 2, flipped=True, offset=14
2 maps to 0, flipped=True, offset=48
0 maps to 1, flipped=True, offset=51
1 maps to 2, flipped=False, offset=-1232
1 maps to 0, flipped=True, offset=50
0 maps to 1, flipped=False, offset=1365
2 maps to 2, flipped=False, offset=152
2 maps to 0, flipped=False, offset=52
0 maps to 1, flipped=True, offset=48
1 maps to 2, flipped=True, offset=1169
0 maps to 0, flipped=False, offset=-41
2 maps to 1, flipped=False, offset=13
1 maps to 2, flipped=True, offset=1333
0 maps to 0, flipped=True, offset=-104
1 maps to 1, flipped=False, offset=-41
2 maps to 2, flipped=True, offset=1179
1 maps to 0, flipped=True, offset=15
0 maps to 1, flipped=False, offset=17
2 maps to 2, flipped=False, offset=-1120
2 maps to 0, flipped=True, offset=-1161
0 maps to 1, flipped=False, offse

In [343]:
def gen_points(report):
    """Generate points from report with co-ordinates relative to scanner 0."""
    for scanner, points in enumerate(report):
        points = iter(points)
        
        # Move through overlaps graph until we get to scanner 0.
        while scanner:
            print(f'{scanner=}')
            points = [transforms[scanner](point) for point in points]
            scanner = overlaps[scanner]
        yield from points

This result lines up with the example.

In [345]:
beacons = set(gen_points(report))
len(beacons)

scanner=1
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=2
scanner=18
scanner=24
scanner=3
scanner=13
scanner=18
scanner=24
scanner=4
scanner=21
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=5
scanner=16
scanner=24
scanner=6
scanner=11
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=7
scanner=11
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=8
scanner=12
scanner=15
scanner=13
scanner=18
scanner=24
scanner=9
scanner=22
scanner=14
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=10
scanner=1
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=11
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=12
scanner=15
scanner=13
scanner=18
scanner=24
scanner=13
scanner=18
scanner=24
scanner=14
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=15
scanner=13
scanner=18
scanner=24
scanner=16
scanner=24
scanner=17
scanner=15
scanner=1

303

In [350]:
def gen_scanner_pos(report):
    """Generate scanner positions from report relative to scanner 0."""
    for scanner, points in enumerate(report):
        points = iter({(0, 0, 0)})
        
        # Move through overlaps graph until we get to scanner 0.
        while scanner:
            print(f'{scanner=}')
            points = [transforms[scanner](point) for point in points]
            scanner = overlaps[scanner]
        yield from points

In [351]:
scanners = set(gen_scanner_pos(report))
scanners

scanner=1
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=2
scanner=18
scanner=24
scanner=3
scanner=13
scanner=18
scanner=24
scanner=4
scanner=21
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=5
scanner=16
scanner=24
scanner=6
scanner=11
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=7
scanner=11
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=8
scanner=12
scanner=15
scanner=13
scanner=18
scanner=24
scanner=9
scanner=22
scanner=14
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=10
scanner=1
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=11
scanner=23
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=12
scanner=15
scanner=13
scanner=18
scanner=24
scanner=13
scanner=18
scanner=24
scanner=14
scanner=17
scanner=15
scanner=13
scanner=18
scanner=24
scanner=15
scanner=13
scanner=18
scanner=24
scanner=16
scanner=24
scanner=17
scanner=15
scanner=1

{(-1343, -2449, 3420),
 (-1339, -89, 5953),
 (-1334, -13, 4750),
 (-1332, 1189, 3446),
 (-1320, 1154, 4747),
 (-1316, -1136, 7025),
 (-1294, -101, 1155),
 (-1283, -1317, 4793),
 (-1281, 1232, 1114),
 (-1186, -1165, 5986),
 (-139, -1192, 3578),
 (-122, -1177, 4698),
 (-118, -2462, 2293),
 (-100, 21, 3416),
 (-98, -1296, 2399),
 (-62, -53, 1206),
 (-48, 69, 2247),
 (-46, -1264, 5830),
 (0, 0, 0),
 (26, -2408, 3603),
 (26, 57, 4708),
 (36, -2327, 4698),
 (1061, -2385, 3500),
 (1101, -2397, 4724),
 (1221, 68, 4633)}

In [363]:
max(
    sum(abs(a[i] - b[i]) for i in range(len(a))) 
    for a, b in itertools.combinations(scanners, r=2)
)

9621

In [ ]:
def manhattan_distances(points):
    """Generates Manhattan distances between pairs of points."""
    
    return {distance(a, b) for a, b in itertools.combinations(points, r=2)}



In [347]:
[i for i, _ in enumerate(report)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

Extract co-ordinate components for each scanner

In [107]:
scanner_0_x, scanner_0_y, scanner_0_z = (set(component) for component in zip(*scanner_0))
scanner_1_x, scanner_1_y, scanner_1_z = (set(dim) for dim in zip(*scanner_1))

- Consider x components of scanner 0 and scanner 1
- The x co-ordinate *could* be the same for each scanner
- But it's unlikely the scanners are in the same position
- So we'll translate scanner 1 by different amounts and calculate the amount of correlation with scanner 0
- Start by translating scanner 1 by the min of scanner 0 minus the max of scanner 1
- This would shift scanner 1 from this:
    ```
                     ---scanner 0 x--->
                                 ---scanner 1 x--->
    ```

    To this:
    ```
                     ---scanner 0 x--->
    ---scanner 1 x--->
    ```
- Increment translation until it is equal to the max of scanner 0 x minus the min of scanner 1 x
    ```
                     ---scanner 0 x--->
                                      ---scanner 1 x--->
    ```
- Then we'll record the translation that results in greatest correlation/overlap

In [82]:
min(scanner_0_x) - max(scanner_1_x)

-1699

In [83]:
max(scanner_0_x) - min(scanner_1_x)

1130

In [125]:
max(similarities(scanner_0_x, scanner_1_x))

(4, 919)

In [126]:
max(similarities(scanner_0_x, scanner_1_y))

(3, -26)

In [128]:
max(similarities(scanner_0_x, scanner_1_z))

(3, 988)

In [127]:
max(similarities(scanner_0_x, {-e for e in scanner_1_x}))

(12, 68)

In [98]:
max(similarities(scanner_0_x, {-e for e in scanner_1_y}), key=lambda x: x[1])

(-347, 3)

In [99]:
max(similarities(scanner_0_x, {-e for e in scanner_1_z}), key=lambda x: x[1])

(-1113, 3)

I take the above as meaning 0x correlates greatest with -1x offset by 68.
Is that correct?

In [100]:
# Scanners 0 and 1 have overlapping detection cubes; 
# the 12 beacons they both detect (relative to scanner 0) are at the following coordinates:
beacons = (
    (-618,-824,-621),
    (-537,-823,-458),
    (-447,-329,318),
    (404,-588,-901),
    (544,-627,-890),
    (528,-643,409),
    (-661,-816,-575),
    (390,-675,-793),
    (423,-701,434),
    (-345,-311,381),
    (459,-707,401),
    (-485,-357,347),
)

In [102]:
for x, y, z in beacons:
    print(-x + 68)

686
605
515
-336
-476
-460
729
-322
-355
413
-391
553


In [ ]:
These same 12 beacons (in the same order) but from the perspective of scanner 1 are:

686,422,578
605,423,415
515,917,-361
-336,658,858
-476,619,847
-460,603,-452
729,430,532
-322,571,750
-355,545,-477
413,935,-424
-391,539,-444
553,889,-390

Yes! That matches!

The high-level algorithm is 

- For each set of components of scanner A's points
- Compute the overlap with components of scanner B's points and the negative of components of scanner B's points for possible translation offsets.
- 

Let's make a function that converts scanner 1's points relative to scanner 0

In [135]:
candidate_dimensions

{0, 1, 2}

In [ ]:


for component in components:

    dimension, flipped, offset, overlap = max(thing(components[0], other_components, candidate_dimensions), key=lambda x: x[3])
    def transform(element):
        return (-element if flipped else element) + offset
    
    transform_
    
    # We've found the dimension to map to, remove it from candidates.
    candidate_dimensions.remove(dimension)
    

- Scanner 0 and scanner 1 do overlap
- Scanner 0's co-ordinates are the reference co-ordinates

Taking scanner 0's x co-ordinates for each point, compute

For each scanner except the first (reference scanner) determine which other scanner shares the greatest number of distances between beacons. This example makes sense because all scanners are in the same position.

In [13]:
scanner, *scanners = parse(example2)

In [19]:
scanner

((-1, -1, 1), (-2, -2, 2), (-3, -3, 3), (-2, -3, 1), (5, 6, -4), (8, 0, 7))

In [5]:
def most_similar(scanner, scanners):
    """Returns most similar scanner in scanners."""
    distances = [distance(scanner) for scanner in scanners]
    return {
        reference: sorted(
            (other for other in range(len(scanners)) if other != reference),
            key=lambda other: len(distances[reference] & distances[other])
        )[0]
        for reference in range(1, len(scanners))
    }

most_similar(parse(example2))

{1: 0, 2: 0, 3: 0, 4: 0}

For each component of reference, find similarity to other components.